In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=940ec3eb8a0f5610be562bd8736d6a98a5b53de34f7359e7a8df17f443ff5d3b
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [ ]:
import torch
import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
# from torch_poly_lr_decay import PolynomialLRDecay
import random


torch.set_num_threads(6)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels):
        super(EfficientNet_MultiLabel, self).__init__()
        self.network = EfficientNet.from_pretrained('efficientnet-b2', in_channels=in_channels)
        self.output_layer1 = nn.Linear(1000, 26)
        # self.dense1_bn = nn.BatchNorm1d(500)
        # self.output_layer2 = nn.Linear(500, 250)
        # self.dense2_bn = nn.BatchNorm1d(250)
        # self.output_layer3 = nn.Linear(250, 26)

    def forward(self, x):
        x = F.relu(self.network(x))
        x = torch.sigmoid(self.output_layer1(x))
        # x = torch.relu(self.output_layer2(x))
        # x = torch.sigmoid(self.output_layer3(x))
        return x

In [ ]:
!mkdir "./dirty_mnist_test"
!unzip "/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/test_dirty_mnist_2nd.zip" -d "./dirty_mnist_test/"

test_imgs_dir = np.array(sorted(glob.glob('./dirty_mnist_test/*')))

test_imgs=[]
for path in tqdm(test_imgs_dir):
    test_img=cv2.imread(path, cv2.IMREAD_COLOR)
    test_imgs.append(test_img)
test_imgs=np.array(test_imgs)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: ./dirty_mnist_test/50000.png  
 extracting: ./dirty_mnist_test/50001.png  
 extracting: ./dirty_mnist_test/50002.png  
 extracting: ./dirty_mnist_test/50003.png  
 extracting: ./dirty_mnist_test/50004.png  
 extracting: ./dirty_mnist_test/50005.png  
 extracting: ./dirty_mnist_test/50006.png  
 extracting: ./dirty_mnist_test/50007.png  
 extracting: ./dirty_mnist_test/50008.png  
 extracting: ./dirty_mnist_test/50009.png  
 extracting: ./dirty_mnist_test/50010.png  
 extracting: ./dirty_mnist_test/50011.png  
 extracting: ./dirty_mnist_test/50012.png  
 extracting: ./dirty_mnist_test/50013.png  
 extracting: ./dirty_mnist_test/50014.png  
 extracting: ./dirty_mnist_test/50015.png  
 extracting: ./dirty_mnist_test/50016.png  
 extracting: ./dirty_mnist_test/50017.png  
 extracting: ./dirty_mnist_test/50018.png  
 extracting: ./dirty_mnist_test/50019.png  
 extracting: ./dirty_mnist_test/50020.png  
 extracting: ./dirty_mnist_test/50021.pn

100%|██████████| 5000/5000 [00:04<00:00, 1211.39it/s]


In [ ]:
test_imgs.shape

(5000, 256, 256, 3)

In [ ]:
# 메모리에서 load
class MnistDataset_v2(Dataset):
    def __init__(self, imgs=None, labels=None, transform=None, train=True):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        self.train=train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get1
        img = self.imgs[idx]
        img = self.transform(img)
        
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img

In [ ]:
# normal

test_transform = transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.5350, 0.5350, 0.5350), (0.3008, 0.3008, 0.3008))
        ])

In [ ]:
predictions_list = []
# 배치 단위로 추론
submission_1_0 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1_60epoch_0227_fold1.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission_1_0.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
100%|██████████| 157/157 [00:13<00:00, 11.25it/s]


In [ ]:
# 배치 단위로 추론
submission_2_0 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1-fold0.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission_2_0.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 157/157 [00:14<00:00, 11.21it/s]


In [ ]:
# horizon flip

test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=1),
        # transforms.Normalize((0.5350, 0.5350, 0.5350), (0.3008, 0.3008, 0.3008))
        ])

In [ ]:
# 배치 단위로 추론
submission_1_1 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1_60epoch_0227_fold1.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission_1_1.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 157/157 [00:31<00:00,  5.00it/s]


In [ ]:
# 배치 단위로 추론
submission_2_1 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1-fold0.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission_2_1.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 157/157 [00:31<00:00,  5.00it/s]


In [ ]:
# vertical flip

test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomVerticalFlip(p=1),
        # transforms.Normalize((0.5350, 0.5350, 0.5350), (0.3008, 0.3008, 0.3008))
        ])

In [ ]:
# 배치 단위로 추론
submission_1_2 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1_60epoch_0227_fold1.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission_1_2.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 157/157 [00:31<00:00,  5.04it/s]


In [ ]:
# 배치 단위로 추론
submission_2_2 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1-fold0.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission_2_2.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 157/157 [00:31<00:00,  5.04it/s]


In [ ]:
# double filp

test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=1),
        transforms.RandomVerticalFlip(p=1),
        # transforms.Normalize((0.5350, 0.5350, 0.5350), (0.3008, 0.3008, 0.3008))
        ])

In [ ]:
# 배치 단위로 추론
submission_1_3 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1_60epoch_0227_fold1.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission_1_3.iloc[n*32:(n+1)*32,1:] += pred_test

Loaded pretrained weights for efficientnet-b2


  0%|          | 0/157 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 157/157 [00:49<00:00,  3.20it/s]


In [ ]:
# 배치 단위로 추론
submission_2_3 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1-fold0.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission_2_3.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 157/157 [00:48<00:00,  3.22it/s]


In [ ]:
predictions_list = []
predictions_list.append(submission_1_0)
predictions_list.append(submission_1_1)
predictions_list.append(submission_1_2)
predictions_list.append(submission_1_3)

predictions_list.append(submission_2_0)
predictions_list.append(submission_2_1)
predictions_list.append(submission_2_2)
predictions_list.append(submission_2_3)

predictions_list

[      index         a             b  ...             x             y         z
 0     50000  1.000000  4.160714e-07  ...  9.989616e-01  4.086220e-07  0.049084
 1     50001  0.573632  6.470272e-01  ...  8.258223e-08  1.901414e-04  0.000079
 2     50002  0.000044  1.058104e-04  ...  4.293344e-06  8.856723e-01  1.000000
 3     50003  0.989270  9.999774e-01  ...  1.000000e+00  9.999983e-01  0.999997
 4     50004  0.224379  4.950597e-01  ...  5.951727e-06  9.287701e-01  0.000035
 ...     ...       ...           ...  ...           ...           ...       ...
 4995  54995  0.001327  2.695559e-02  ...  1.389849e-01  1.000000e+00  0.953627
 4996  54996  1.000000  3.685665e-01  ...  9.999999e-01  1.098098e-06  1.000000
 4997  54997  0.999967  3.129888e-03  ...  4.629958e-06  5.717273e-04  1.000000
 4998  54998  0.977726  3.251634e-06  ...  1.163608e-04  1.375892e-03  0.912654
 4999  54999  0.009011  9.731693e-01  ...  6.553367e-01  1.000000e+00  0.997957
 
 [5000 rows x 27 columns],
       inde

In [ ]:
df_concat = pd.concat((submission_1_0, submission_1_1,submission_1_2,submission_1_3, \
                       submission_2_0, submission_2_1, submission_2_2, submission_2_3))

by_row_index = df_concat.groupby(df_concat.index)
df_means = by_row_index.mean()

df_means

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,0.999996,0.000321,1.000000,0.018862,0.999997,0.999999,0.000061,0.999998,0.999899,0.002380,0.001157,0.389369,0.376756,0.924964,0.002886,0.989935,0.070847,0.008374,0.000017,0.820599,0.999994,0.130288,0.062207,0.999246,0.001365,0.278920
1,50001,0.125723,0.421306,0.671365,0.078805,1.000000,0.000057,0.750106,0.034368,0.999974,0.353794,0.936348,0.917367,0.001815,0.001107,0.999987,0.999973,0.032936,0.143390,0.024741,0.001005,0.999993,0.987905,0.632283,0.000753,0.194514,0.002056
2,50002,0.012379,0.032076,0.143651,0.888412,0.999821,0.146875,1.000000,0.361045,1.000000,0.017502,0.306985,0.015515,0.001313,0.196251,0.000033,1.000000,0.008777,0.987534,0.999996,0.005838,0.999959,0.019436,0.999999,0.001195,0.713592,1.000000
3,50003,0.975405,0.999961,0.017432,0.251881,0.056338,0.991221,0.512004,0.000223,0.054298,0.118398,0.999947,0.999948,0.999998,0.000302,0.000006,0.858629,0.409018,0.271462,0.001705,0.815884,0.002238,0.782737,0.000210,0.999998,0.824082,0.999986
4,50004,0.042018,0.074442,0.999980,0.020726,0.999999,0.999992,0.001732,0.000202,0.000087,0.000091,0.000253,0.234102,1.000000,0.022868,0.999997,0.038778,0.999996,0.069103,0.999966,0.999987,0.999667,0.577023,0.999996,0.003131,0.397337,0.004126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0.013199,0.757787,1.000000,0.322193,0.000150,0.009934,0.185935,1.000000,0.013392,0.952577,0.441592,0.806558,0.392524,0.056567,0.994441,0.003224,0.023470,0.996339,0.001007,0.999210,0.599055,0.293263,0.999944,0.020482,1.000000,0.778438
4996,54996,0.999990,0.075328,0.999829,0.262995,0.999669,0.063342,0.239050,0.001987,0.998814,0.999998,0.000210,0.002308,0.009683,0.300022,1.000000,1.000000,0.975473,0.000820,0.468326,0.371320,0.049526,0.006342,0.048367,0.878060,0.000050,1.000000
4997,54997,0.992016,0.001909,0.003547,1.000000,0.494561,1.000000,0.000187,0.011342,0.124182,0.211377,0.999984,0.991123,0.875482,0.493863,0.999799,0.999503,0.000173,0.199817,0.001308,0.511555,0.999990,0.999992,0.952367,0.000013,0.000556,0.999880
4998,54998,0.480860,0.163552,0.999951,0.050015,0.000905,0.055780,0.470485,0.027068,0.976977,0.999751,0.039871,0.276816,0.159752,0.347674,0.000325,0.002345,1.000000,0.876428,0.500492,0.514624,0.015661,0.019999,0.999782,0.009164,0.016290,0.503845


In [ ]:
# 평균 값이 0.5보다 클 경우 1 작으면 0
predictions_mean = (df_means > 0.5) * 1
predictions_mean.iloc[:,0] = df_means.iloc[:,0]
predictions_mean

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,1,0,1,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,0
1,50001,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,0,1,1,1,0,0,0
2,50002,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,1,1
3,50003,1,1,0,0,0,1,1,0,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,1,1,1
4,50004,0,0,1,0,1,1,0,0,0,0,0,0,1,0,1,0,1,0,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,1,1,0,1,0,1,1
4996,54996,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1
4997,54997,1,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,0,0,0,1,1,1,1,0,0,1
4998,54998,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1


In [ ]:
predictions_mean.to_csv("eff_net_b2_epoch60_TTA.csv", index = False)